In [ ]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
!nvidia-smi

Wed May 15 05:03:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8              14W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip -q install langchain==0.0.343
!pip -q install bitsandbytes
!pip -q install torch
!pip -q install git+https://github.com/huggingface/transformers@v4.31-release
!pip -q install -U git+https://github.com/huggingface/peft.git
!pip -q install accelerate
!pip -q install loralib
!pip -q install einops
!pip -q install sentencepiece==0.1.99
!pip install -q datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.2/188.2 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
     ━

In [ ]:
import json
import os
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from transformers import pipeline
from datasets import load_dataset
from langchain import HuggingFacePipeline
from langchain.memory import ConversationTokenBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from google.cloud import translate_v2 as translate

from huggingface_hub import HfFolder
HfFolder.save_token('enter_your_token_here')

In [ ]:
PEFT_MODEL = "muhammadhasnain100/bio_llama-2_13b"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
added 22 packages from 22 contributors in 2.371s


In [ ]:
from flask import Flask, request, jsonify
import uuid
import threading
app = Flask(__name__)

# For Local system
# !export GOOGLE_APPLICATION_CREDENTIALS="/content/hadeeth-ocr-project-240f71b66850.json"

# Path to the service account key file on Colab
service_account_path = '/content/chat-bot-abfba-0d6b06e93426.json'

# Set the environment variable
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_account_path

translate_client = translate.Client()

def translate_text(text, source_lang, target_language='en'):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    """
    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    translation = translate_client.translate(text, target_language=target_language, source_language=source_lang)
    return translation['translatedText']

# Initialize the pipeline with appropriate parameters
def initialize_pipeline(model, tokenizer):
  return pipeline("text-generation",
                  model=model,
                  tokenizer=tokenizer,
                  torch_dtype=torch.bfloat16,
                  device_map="auto",
                  max_new_tokens=256,
                  do_sample=True,
                  top_k=10,
                  top_p=0.7,
                  num_return_sequences=1,
                  eos_token_id=tokenizer.eos_token_id)

# Template for prompt
template = """
<s>[INST] <<SYS>>
You are an expert in generating medical-related responses. Your task is to answer the user's query in a professional and articulate manner.
Ensure that your response is concise, precise, and directly addresses the query at hand. Provide short responses, make sure your response should not exceed 150 words. It should contain only relevant information,
avoiding any superfluous or unrelated content. When appropriate, include authoritative references or links to offer additional, credible
information or when specifically requested by the user. Your answer should demonstrate expertise and be structured to provide clear,
understandable, and useful information.
<</SYS>>
Chat_history: {chat_history}
Question: {question} [/INST]
""".strip()


# Initialize the large language model (LLM) pipeline
def initialize_llm(model, tokenizer):
  pipe = initialize_pipeline(model, tokenizer)
  return HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature': 0.1})

# Create a new chain for interaction
def create_new_chain(prompt_template, llm):
  prompt = PromptTemplate(template=prompt_template, input_variables=["chat_history", "question"])
  memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=1500, memory_key="chat_history")
  return LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=False)

# Delete a chain using its id
def delete_chain(chains_dict, chain_id):
  if chain_id in chains_dict:
      del chains_dict[chain_id]
  print('Chat Successfully Deleted')

def count_tokens(text):
    # Use the tokenizer to encode the text and return the token count
    tokens = tokenizer.encode(text, add_special_tokens=True)
    return len(tokens)

# Get response
def response(chains, id, query):
    try:
        # Clear CUDA cache and collect garbage (optional)
        torch.cuda.empty_cache()
        gc.collect()

        count =  count_tokens(query)
        if count > 1024:
          return "Input query is too Long."
        else:
          # Attempt to get the chain and generate a response
          chain = chains.get(id)
          if chain is None:
              return "Chain not found for the given ID."

          response = chain.predict(question=query)
          return response
    except Exception as e:
        # Handle any exceptions that occur
        return f"An error occurred and Try to Start New Chat"

llm = initialize_llm(model, tokenizer)
chains = {}

@app.route('/newchat', methods=['POST'])
def new_chat():
    global chains
    new_chain_id = str(uuid.uuid4())
    chains[new_chain_id] = create_new_chain(template, llm)
    return jsonify({'chat_id': new_chain_id})

@app.route('/deletechat', methods=['POST'])
def delete_chat():
    chat_id = request.json.get('chat_id')
    if chat_id in chains:
        delete_chain(chains, chat_id)
        return jsonify({'status': 'success', 'message': 'Chat deleted successfully'})
    else:
        return jsonify({'status': 'error', 'message': 'Chat ID not found'}), 404

@app.route('/response', methods=['POST'])
def get_response():
    chat_id = request.json.get('chat_id')
    query = request.json.get('query')
    lang_code = request.json.get('lang_code')

    if lang_code != 'en':
        query = translate_text(query, lang_code, 'en')

    # Proceed with the rest of the function if token count does not exceed 512
    if chat_id in chains:
        chat_response = response(chains, chat_id, query)

        if lang_code != 'en':
          chat_response = translate_text(chat_response, 'en', lang_code)

        return jsonify({'response': chat_response})
    else:
        return jsonify({'status': 'error', 'message': 'Chat ID not found'}), 404

# # Function to run the Flask app
def run_flask():
  app.run(port=5000)

# Start the Flask app in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.start()
!lt --port 5000


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


your url is: https://tangy-lines-write.loca.lt


INFO:werkzeug:127.0.0.1 - - [15/May/2024 05:14:20] "POST /newchat HTTP/1.1" 200 -
